In [ ]:
from __future__ import unicode_literals # 이게 뭘까
import requests
import pymongo
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from scrapy.http import TextResponse
import urllib.request
import urllib
import getpass
import time
import re
# from google.cloud import vision
# from google.cloud.vision import types
import os
import io
from os import walk
from bs4 import BeautifulSoup

In [ ]:
!cd

In [ ]:
if not os.path.exists('./thumb_detail'):
    os.mkdir('./thumb_detail')

webtoon_info = []
total_df_info = []
total_img_count = 0

domain_url = 'https://comic.naver.com'
main_url = 'https://comic.naver.com/webtoon/weekday.nhn'

html = requests.get(main_url).text
soup = BeautifulSoup(html, 'html.parser')
webtoon_area = soup.select('#content > div.list_area.daily_all > div.col > div > ul > li > div.thumb > a')

for each_webtoon in webtoon_area:
    webtoon_last_page = domain_url + each_webtoon['href'] + '&page={}'.format(99999)
    html = requests.get(webtoon_last_page).text
    soup = BeautifulSoup(html, 'html.parser')
    last_page_num = int(soup.select('#content > div.paginate > div > strong.page')[0].find('em').text)
    
    title = soup.select('.comicinfo h2')[0].text.split('\n')[1]
#     title = ', '.join([each_ele.strip() for each_ele in soup.select('.comicinfo h2')[0].text.split('\n') if each_ele != ''])
    title = ', '.join(['&'.join([each_text.strip() for each_text in each_text.split('/')]) for each_text in title.split(',')])
    excel_title = title
    title = re.sub('[-=+#/\?:^$.@*\"※~%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', title)
#     print(title)
    
    author = soup.select('.comicinfo h2')[0].text.split('\n')[2] 
#     author = ', '.join([each_ele.strip() for each_ele in soup.select('.comicinfo h2')[0].text.split('\n') if each_ele != ''])
    author = ', '.join([','.join([each_text.strip() for each_text in each_text.split('/')]) for each_text in author.split(',')])
    author = re.sub('[-=+#/\?:^$.@*\"※~%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', author)
    print(author)
#     print(type(author))
#     print(author.split(','))
    
    if len(author.split(','))>0:
        writer = author.split(',')[0]
    else:
        writer = author
#     print(writer)
    
    if len(author.split(','))>0:
        illustrator = author.split(',')[-1]
    else:
        illustrator = author
#     print(illustrator)


    genre = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.genre')[0].text
    story = soup.select('#content > div.comicinfo > div.detail > p:nth-child(2)')[0].text
    age = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.age')[0].text

    webtoon_info.append({'title': excel_title, 'writer': writer, 'illustrator':illustrator, 'genre': genre, 'story': story, 'age': age})

    for page_num in range(1, last_page_num + 1):
        webtoon_url = domain_url + each_webtoon['href'] + '&page={}'.format(page_num)
        html = requests.get(webtoon_url).text
        soup = BeautifulSoup(html, 'html.parser')

        if not os.path.exists('./thumb_detail/{}'.format(illustrator)):
            os.mkdir('./thumb_detail/{}'.format(illustrator))

        thumbnail_area = soup.find('table', 'viewList')
        thumbnail_img_list = thumbnail_area.select('td > a > img')[1:]
        
#         for each_img_tag in thumbnail_img_list:
#             print(re.sub('[^0-9a-zA-Zㄱ-힗]','',each_img_tag['alt'].split('/')[0]))


        for each_img_tag in thumbnail_img_list:
            urllib.request.urlretrieve(each_img_tag['src'], './thumb_detail/{}/{}.png'.format(illustrator, title + '_' + re.sub('[\\/:*?"<>|]','',str(each_img_tag['alt'].split('/')[0]))))
#             urllib.request.urlretrieve(each_img_tag['src'], './total_img/{}.png'.format(author + '_' + '&'.join(each_img_tag['alt'].split('/')[0])))
            total_df_info.append({'title': excel_title, 'writer': writer,'illustrator':illustrator, 'genre': genre, 'story': story, 'age': age})
            total_img_count += 1
        
    print(title+' done')
            

df = pd.DataFrame(webtoon_info, columns=['title', 'writer', 'illustrator', 'genre', 'story', 'age'])
total_df = pd.DataFrame(total_df_info, columns=['title', 'writer', 'illustrator', 'genre', 'story', 'age'])
df.to_csv('webtoon_info.csv')
df.to_csv('webtoon_info_encode.csv', encoding='utf-8-sig')
total_df.to_csv('total_img_info.csv')

In [32]:
total_img_count

31028